## Import Packages
* gym - collection of environments for reinforcement learning algorithms.
* numpy - package for scientific computing with Python. 
* random - implements pseudo-random number generators for various distributions.
* math -  mathematical functions defined by the C standard.

In [1]:
import gym
import numpy as np
import math

## Define the environment
A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pole starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center.


In [2]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


## Define the environment related constants.
*  Every environment comes with an action_space and an observation_space. These attributes are of type Space, and they describe the format of valid actions and observations. The Discrete space allows a fixed range of non-negative numbers, so in this case valid actions are either 0 or 1. The Box space represents an n-dimensional box, so valid observations will be an array of 4 numbers.
* Define the following constants:
   * Number of discrete states (bucket) per state dimension :position,velocity,angle between the pole and the vertical axis and the angular velocity. 
   * Number of discrete actions (left, right) 
   * Bounds for each discrete state
* Define the minimum exploration and learning rate values.
   

In [3]:
env.action_space.n

2

In [4]:
env.observation_space

Box(4,)

In [5]:
env.observation_space.low 

array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32)

In [6]:
env.observation_space.high

array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],
      dtype=float32)

In [7]:
NUM_BUCKETS = (1, 1, 6, 3)

In [8]:
NUM_ACTIONS = env.action_space.n

In [9]:
STATE_BOUNDS = list(zip(env.observation_space.low, env.observation_space.high))

In [10]:
STATE_BOUNDS[1] = [-0.5, 0.5]

In [11]:
STATE_BOUNDS[3] = [-math.radians(50), math.radians(50)]

In [12]:
STATE_BOUNDS

[(-4.8, 4.8),
 [-0.5, 0.5],
 (-0.41887903, 0.41887903),
 [-0.8726646259971648, 0.8726646259971648]]

## Create Q-Table
* Create a Q-Table for each state-action pair
* Define the learning related constants:
* The learning rate must decay but not too fast. The conditions for convergence are the following:
    *sum(alpha(t), 1, inf) = inf
    *sum(alpha(t)^2, 1, inf) < inf
    * Something like alpha = k/(k+t) can work well.

In [15]:
q_table = np.zeros(NUM_BUCKETS + (NUM_ACTIONS,))

In [16]:
q_table.shape

(1, 1, 6, 3, 2)

In [17]:
print(q_table)

[[[[[0. 0.]
    [0. 0.]
    [0. 0.]]

   [[0. 0.]
    [0. 0.]
    [0. 0.]]

   [[0. 0.]
    [0. 0.]
    [0. 0.]]

   [[0. 0.]
    [0. 0.]
    [0. 0.]]

   [[0. 0.]
    [0. 0.]
    [0. 0.]]

   [[0. 0.]
    [0. 0.]
    [0. 0.]]]]]


In [18]:
EXPLORE_RATE_MIN = 0.01

In [19]:
LEARNING_RATE_MIN = 0.1

In [20]:
def get_explore_rate(t):
    return max(EXPLORE_RATE_MIN, min(1, 1.0 - math.log10((t+1)/25)))

In [21]:
def get_learning_rate(t):
    return max(LEARNING_RATE_MIN, min(0.5, 1.0 - math.log10((t+1)/25)))

## Define method for selecting an action
* Select a random action or the action with the highest q based on explore rate

In [22]:
def select_action(state, explore_rate):
    if random.random() < explore_rate:
        action = env.action_space.sample()
    else:
        action = np.argmax(q_table[state])
    return action

## Define method to define states
* Discretize the continuous dimensions to a number of buckets. 
* Map the state bounds to the bucket array

In [18]:
def state_to_bucket(state):
    
    bucket_indices = []
    
    for i in range(len(state)):
        if state[i] <= STATE_BOUNDS[i][0]:
            bucket_index = 0
            
        elif state[i] >= STATE_BOUNDS[i][1]:
            bucket_index = NUM_BUCKETS[i] - 1
            
        else:
            bound_width = STATE_BOUNDS[i][1] - STATE_BOUNDS[i][0]
            
            offset = (NUM_BUCKETS[i] - 1) * STATE_BOUNDS[i][0] / bound_width
            scaling = (NUM_BUCKETS[i] - 1) / bound_width
            
            bucket_index = int(round(scaling * state[i] - offset))
            
        bucket_indices.append(bucket_index)
    return tuple(bucket_indices)

## Define method for simulation
* Get the initial learning and explore rates
* Intialise discount factor and number of streaks
* Train over 1000 episodes. In each episode, do the following:
* Start the process using the reset method
* Get the initial state by passing in the current observations about the environment
* Iterate over 250 timesteps:
    * Invoke the render method to visualise the episode.
    * Execute a selected action using the step emthod and read the values of reward, observation, done and info.
    * Identify the resultant state from the observations.
    * Identify the state with the highest possible reward from the q table.
    * Compute the q value and update the q table : Q[s, a] = Q[s, a] + alpha*(R + gamma*Max[Q(s’, A)] - Q[s, a])
    
    * Set the new state as current state.
    * If done equals true, episode has terminated.Reset the environment again.
    * The agent wouldn't need more than 200 timesteps to train. Increment the no_streaks value so we can terminate training in 120 episodes, each of no more than 200 timesteps.
    * Update the parameters so the learning and exploration rates decay as the episodes increase
    

In [227]:
def simulate():

    learning_rate = get_learning_rate(0)
    explore_rate = get_explore_rate(0)

    discount_factor = 0.99  
    num_streaks = 0

    for episode in range(1000):
        
        observ = env.reset()
    
        state_0 = state_to_bucket(observ)

        for t in range(250):

            env.render()
            
            action = select_action(state_0, explore_rate)
            
            observ, reward, done, _ = env.step(action)

            state = state_to_bucket(observ)
            
            best_q = np.amax(q_table[state])
            
            q_table[state_0 + (action,)] +=\
                learning_rate * (reward + discount_factor*(best_q) - q_table[state_0 + (action,)])

            state_0 = state

            print("\nEpisode = %d" % episode)
            print("t = %d" % t)
            print("Action: %d" % action)
            print("State: %s" % str(state))
            print("Reward: %f" % reward)
            print("Best Q: %f" % best_q)
            print("Explore rate: %f" % explore_rate)
            print("Learning rate: %f" % learning_rate)
            print("Streaks: %d" % num_streaks)

            print("")

            if done:
                print("Episode %d finished after %f time steps" % (episode, t))
                if (t >= 199):
                    num_streaks += 1
                else:
                    num_streaks = 0
                break

        if num_streaks > 120:
            break
      
        explore_rate = get_explore_rate(episode)
        learning_rate = get_learning_rate(episode)

## Train the agent
* Invoke the simulate method
* Close the rendered environment.
* Check the values in q talbe.

In [228]:
simulate()


Episode = 0
t = 0
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 0.000000
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 1
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 0.500000
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 2
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 0.747500
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 3
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 1.120013
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 4
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 1.428156
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 5
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 1.921015
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 6
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 2.010909
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episo


Episode = 3
t = 7
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 0.000000
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 8
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 3.061669
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 9
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 3.061669
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 10
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 3.546361
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 11
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 4.028629
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 12
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 4.028629
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 13
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 2.881792
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


E


Episode = 5
t = 16
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 8.306926
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 5
t = 17
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 8.702990
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 5
t = 18
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 8.961443
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 5
t = 19
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 9.287409
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 5
t = 20
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 9.577989
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 5
t = 21
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 9.884809
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 5
t = 22
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 10.335385
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 8
t = 3
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 12.806507
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 8
t = 4
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 13.242474
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 8
t = 5
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 13.455196
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 8
t = 6
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 13.781559
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 8
t = 7
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 7.961003
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 8
t = 8
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 7.961003
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 8
t = 9
Action: 0
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 7.961003
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


E


Episode = 11
t = 2
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 15.493038
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 3
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 15.809667
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 4
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 16.072304
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 5
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 14.186185
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 6
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 16.072304
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 7
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 16.072304
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 8
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 16.491943
Explore rate: 1.000000
Learning rate: 0.500000
Stre

...


Episode = 140
t = 241
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 65.411147
Explore rate: 0.251812
Learning rate: 0.251812
Streaks: 93

Episode 140 finished after 241.000000 time steps

Episode = 140
t = 242
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 65.498246
Explore rate: 0.251812
Learning rate: 0.251812
Streaks: 94

Episode 140 finished after 242.000000 time steps

Episode = 140
t = 243
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 65.585126
Explore rate: 0.251812
Learning rate: 0.251812
Streaks: 95

Episode 140 finished after 243.000000 time steps

Episode = 140
t = 244
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 65.585126
Explore rate: 0.251812
Learning rate: 0.251812
Streaks: 96

Episode 140 finished after 244.000000 time steps

Episode = 140
t = 245
Action: 0
State: (0, 0, 2, 2)
Reward: 1.000000
Best Q: 65.441851
Explore rate: 0.251812
Learning rate: 0.251812
Streaks: 97

Episode 140 finished after 245.000000 time steps

Episode = 140



Episode = 141
t = 55
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 66.551943
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 56
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 66.551943
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 57
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 66.635136
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 58
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 65.695811
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 59
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 66.580644
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 60
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 66.580644
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 61
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 66.663765
Explore rate: 0.248721
Le


Episode = 141
t = 119
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 67.358589
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 120
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 67.439775
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 121
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 67.014233
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 122
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 67.268493
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 123
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 67.268493
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 124
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 67.349903
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 125
Action: 0
State: (0, 0, 3, 2)
Reward: 1.000000
Best Q: 66.810220
Explore rate: 0.24


Episode = 141
t = 182
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 68.034363
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 183
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 68.034363
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 184
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 68.113868
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 185
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 68.113868
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 186
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 68.193175
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 187
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 67.844365
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 102


Episode = 141
t = 188
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 68.290762
Explore rate: 0.24


Episode = 141
t = 232
Action: 0
State: (0, 0, 2, 2)
Reward: 1.000000
Best Q: 68.712378
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 135

Episode 141 finished after 232.000000 time steps

Episode = 141
t = 233
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 68.524656
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 136

Episode 141 finished after 233.000000 time steps

Episode = 141
t = 234
Action: 0
State: (0, 0, 3, 2)
Reward: 1.000000
Best Q: 67.429376
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 137

Episode 141 finished after 234.000000 time steps

Episode = 141
t = 235
Action: 0
State: (0, 0, 3, 2)
Reward: 1.000000
Best Q: 67.429376
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 138

Episode 141 finished after 235.000000 time steps

Episode = 141
t = 236
Action: 1
State: (0, 0, 3, 2)
Reward: 1.000000
Best Q: 67.429376
Explore rate: 0.248721
Learning rate: 0.248721
Streaks: 139

Episode 141 finished after 236.000000 time steps

Episode =

In [229]:
env.close()

In [230]:
print(q_table)

[[[[[ 0.          0.        ]
    [ 0.          0.        ]
    [ 0.          0.        ]]

   [[30.30637202 33.37867099]
    [ 5.77801176 24.95325648]
    [ 0.          0.        ]]

   [[68.08267828 56.72551123]
    [69.09711054 68.02461963]
    [64.45902239 68.7439736 ]]

   [[68.77855073 65.64067919]
    [68.10945104 68.77116879]
    [61.63491999 67.84094155]]

   [[ 0.          0.        ]
    [21.72750622 18.79018525]
    [21.37016986 19.30913702]]

   [[ 0.          0.        ]
    [ 0.          0.        ]
    [ 0.          0.        ]]]]]
